In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/30days-folds/train_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

In [ ]:
df

In [ ]:
features = [i for i in df.columns if i not in ('id', 'target', 'kfold')]
features

In [ ]:
cat_column = [column for column in features if 'cat' in column]
cat_column

In [ ]:
test = df_test[features]

In [ ]:
final_predictions = []
for fold in range(5):
    x_train = df[df.kfold != fold].reset_index(drop = True)
    x_valid = df[df.kfold == fold].reset_index(drop = True)
    xtest = test.copy()
    
    
    y_train = x_train.target
    y_valid = x_valid.target
    
    x_train = x_train[features]
    x_valid = x_valid[features]
    
    encoder = OrdinalEncoder()
    x_train[cat_column] = encoder.fit_transform(x_train[cat_column])
    x_valid[cat_column] = encoder.transform(x_valid[cat_column])
    xtest[cat_column] = encoder.transform(xtest[cat_column])
    
    model = XGBRegressor(random_state=fold, n_jobs=4)
    model.fit(x_train, y_train)
    prediction = model.predict(x_valid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    mae = mean_absolute_error(prediction, y_valid)
    print("mean absolute error of fold",fold+1,"is", mae)
    

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
preds

In [ ]:
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)